step52: GPU対応

52.1 Cupyのインストールと使用法

In [1]:
import cupy as cp

x = cp.arange(6).reshape(2, 3)
print(x)

y = x.sum(axis=1)
print(y)

[[0 1 2]
 [3 4 5]]
[ 3 12]


In [2]:
import numpy as np

# numpy -> cupy
n = np.array([1, 2, 3])
c = cp.asarray(n)
assert type(c) == cp.ndarray

# cupy -> numpy
c = cp.array([1, 2, 3])
n = cp.asnumpy(c)
assert type(n) == np.ndarray

In [3]:
# npとcpのデータに応じてxpにモジュール(np, cp)を代入する->モジュールが不明な時確認して両方に対応できる
x = np.array([1, 2, 3])
xp = cp.get_array_module(x)
assert xp == np

x = cp.array([1, 2, 3])
xp = cp.get_array_module(x)
assert xp == cp

52.2 cudaモジュール

In [4]:
# Cupyがインストールされていなくてもエラーを出さず代わりにNumpyを使えるようにする
import numpy as np
gpu_enable = True
try:
    import cupy as cp
    cupy = cp
except ImportError:
    gpu_enable = False
import sys
sys.path.append(r'C:\Users\jiwon\OneDrive\Desktop\DL3')
from dezero import Variable

# Variableにも対応したCupyとNumpy両対応のメソッドを定義

# 対応したモジュールを返す
def get_array_module(x):
    if isinstance(x, Variable):
        x = x.data

    if not gpu_enable: # インポート時にCupyがインストールされていないならNumpyモジュールを返す
        return np
    xp = cp.get_array_module(x)
    return xp

# numpy配列に変換する
def as_numpy(x):
    if isinstance(x, Variable):
        x = x.data

    if np.isscalar(x): # numpyスカラをnumpy配列に
        return np.array(x)
    elif isinstance(x, np.ndarray): # numpy配列はそのまま
        return x
    return cp.asnumpy(x) # Cupy配列はnumpy配列に

# Cupy配列に変換する
def as_cupy(x):
    if isinstance(x, Variable):
        x = x.data

    if not gpu_enable:
        raise Exception('Cupy cannot be loaded. Install CuPy!')
    return cp.asarray(x)

52.3 Variable/Layer/DataLoaderクラスの追加実装

In [5]:
# dezero/core.py
import dezero

try:
    import cupy
    array_types = (np.ndarray, cupy.ndarray)
except ImportError:
    array_types = (np.ndarray)

class Variable:
    def __init__(self, data, name=None):
        if data is not None:
            if not isinstance(data, array_types):
                raise TypeError('{} is not supported'.format(type(data)))

    def backward(self, retain_grad=False, create_graph=False):
        if self.grad is None:
            xp = dezero.cuda.get_array_module(self.data)
            self.grad = Variable(xp.ones_like(self.data))

In [1]:
# dezero/core.py
# class Variable
def to_cpu(self):
    if self.data is not None:
        self.data = dezero.cuda.as_numpy(self.data)

def to_gpu(self):
    if self.data is not None:
        self.data = dezero.cuda.as_cupy(self.data)

In [2]:
# dezero/layers.py
# class Layer
def to_cpu(self):
    for param in self.params():
        param.to_cpu()

def to_gpu(self):
    for param in self.params():
        param.to_gpu()

In [4]:
# dezero/dataloader.py
# class DataLoader
import math
import sys
sys.path.append(r'C:\Users\jiwon\OneDrive\Desktop\DL3')
from dezero import cuda

def __init__(self, dataset, batch_size, shuffle=True, gpu=False):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.data_size = len(dataset)
        self.max_iter = math.ceil(self.data_size / batch_size)
        self.gpu = gpu

        self.reset()

def __next__(self):
        if self.iteration >= self.max_iter:
                self.reset()
                raise StopIteration

        i, batch_size = self.iteration, self.batch_size
        batch_index = self.index[i * batch_size : (i + 1) * batch_size]
        batch = [self.dataset[i] for i in batch_index]
        xp = cuda.cupy if self.gpu else np
        x = xp.array([example[0] for example in batch])
        t = xp.array([example[1] for example in batch])

        self.iteration += 1
        return x, t

def to_cpu(self):
        self.gpu = False

def to_gpu(self):
        self.gpu = True

# dezero/functions内における全てのnp.xxx()において以下のように置き換える
xp = cuda.get_array_module(x)
y = xp.sin(x)

# dezero/core.py
def as_array(x, array_module=np):
    if np.isscalar(x):
        return array_module.array(x)

def add(x0, x1):
    x1 = as_array(x1, dezero.cuda.get_array_module(x0.data))
    return Add()(x0, x1)

def mul(x0, x1):
    x1 = as_array(x1, dezero.cida.get_array_module(x0.data))
    return Mul()(x0, x1)

# sub, rsub, div, rdivでも同じ修正を行う

step52.5 GPUでMNISTを学習

In [2]:
import time
import sys
sys.path.append(r'C:\Users\jiwon\OneDrive\Desktop\DL3')
import dezero
import dezero.functions as F
from dezero import optimizers
from dezero import DataLoader
from dezero.models import MLP


max_epoch = 5
batch_size = 100

train_set = dezero.datasets.MNIST(train=True)
train_loader = DataLoader(train_set, batch_size)
model = MLP((1000, 10))
optimizer = optimizers.SGD().setup(model)


# GPU mode
if dezero.cuda.gpu_enable:
    train_loader.to_gpu()
    model.to_gpu()

for epoch in range(max_epoch):
    start = time.time()
    sum_loss = 0

    for x, t in train_loader:
        y = model(x)
        loss = F.softmax_cross_entropy(y, t)
        model.cleargrads()
        loss.backward()
        optimizer.update()
        sum_loss += float(loss.data) * len(t)

    elapsed_time = time.time() - start
    print('epoch: {}, loss: {:.4f}, time: {:.4f}[sec]'.format(
        epoch + 1, sum_loss / len(train_set), elapsed_time))

epoch: 1, loss: 1.9478, time: 3.0246[sec]
epoch: 2, loss: 1.3087, time: 2.8175[sec]
epoch: 3, loss: 0.9371, time: 2.7932[sec]
epoch: 4, loss: 0.7463, time: 2.8201[sec]
epoch: 5, loss: 0.6386, time: 2.8389[sec]


stpe53: モデルの保存と読み込み

53.1 Numpyのsave関数とload関数

In [2]:
import numpy as np

x = np.array([1,2,3])
np.save('test.npy', x)

x = np.load('test.npy')
print(x)

[1 2 3]


In [5]:
x1 = np.array([1, 2, 3])
x2 = np.array([4, 5, 6])
np.savez('test2.npz', x1=x1, x2=x2)

arrays = np.load('test2.npz')
x1 = arrays['x1']
x2 = arrays['x2']
print(x1)
print(x2)

[1 2 3]
[4 5 6]


In [7]:
x1 = np.array([1, 2, 3])
x2 = np.array([4, 5, 6])
data = {'x1':x1, 'x2':x2}

np.savez('test3.npz', **data)

arrays = np.load('test2.npz')
x1 = arrays['x1']
x2 = arrays['x2']
print(x1)
print(x2)

[1 2 3]
[4 5 6]


53.2 Layerクラスのパラメータをフラットに

In [8]:
# dezero/layers.py
# class Layer:

# params_dict: 平坦化したパラメータを保持する辞書 parent_key: 親レイヤの名前（一番上のレイヤには親はいないが次の層からはkeyが代入される）
def _flatten_params(self, params_dict, parent_key=""):
    for name in self._params:
        obj = self.__dict__[name] # 今参照しているレイヤのディクショナリから_params(子レイヤかパラメータ)を読んでobjにする
        key = parent_key + '/' + name if parent_key else name # ディクショナリのキーを作成 e.g. 'p2', 'l1/p1'

        if isinstance(obj, Layer): # オブジェクトがレイヤーならぱその中にあるパラメータが平坦化され取り出される
            obj._flatten_params(params_dict, key)
        else: # パラメータならparams_dictに追加していく（平坦化）
            params_dict[key] = obj

In [9]:
import sys
sys.path.append(r'C:\Users\jiwon\OneDrive\Desktop\DL3')
from dezero import Layer, Parameter

layer = Layer()

l1 = Layer()
l1.p1 = Parameter(np.array(1))

layer.l1 = l1
layer.p2 = Parameter(np.array(2))
layer.p3 = Parameter(np.array(3))

params_dict = {}
layer._flatten_params(params_dict)
print(params_dict)

{'p2': variable(2), 'p3': variable(3), 'l1/p1': variable(1)}


53.3 Layerクラスのsave関数とload関数

In [1]:
# dezero/layers.py
# Layer
import os

def save_weights(self, path):
    self.to_cpu()

    params_dict = {}
    self._flatten_params(params_dict)
    array_dict = {key: param.data for key, param in params_dict.items() if param is not None}

    try:
        np.savez_compressed(path, **array_dict)
    except (Exception, KeyboardInterrupt) as e:
        if os.path.exists(path):
            os.remove(path)
        raise

def load_weights(self, path):
    npz = np.load(path)
    params_dict = {}
    self._flatten_params(params_dict)
    for key, param in params_dict.items():
        param.data = npz[key]

In [4]:
import sys, os
sys.path.append(r'C:\Users\jiwon\OneDrive\Desktop\DL3')
import dezero
import dezero.functions as F
from dezero import optimizers
from dezero import DataLoader
from dezero.models import MLP


max_epoch = 3
batch_size = 100

train_set = dezero.datasets.MNIST(train=True)
train_loader = DataLoader(train_set, batch_size)
model = MLP((1000, 10))
optimizer = optimizers.SGD().setup(model)

# もしファイルがあればパラメータをロードして上書き、なければ上でランダムに設定された初期値をそのまま使用
if os.path.exists('my_mlp.npz'):
    model.load_weights('my_mlp.npz')

# GPU mode
if dezero.cuda.gpu_enable:
    train_loader.to_gpu()
    model.to_gpu()

for epoch in range(max_epoch):
    sum_loss = 0

    for x, t in train_loader:
        y = model(x)
        loss = F.softmax_cross_entropy(y, t)
        model.cleargrads()
        loss.backward()
        optimizer.update()
        sum_loss += float(loss.data) * len(t)

    print('epoch: {}, loss: {:.4f}'.format(
        epoch + 1, sum_loss / len(train_set)))

model.save_weights('my_mlp.npz')

epoch: 1, loss: 0.4444
epoch: 2, loss: 0.4284
epoch: 3, loss: 0.4154


In [5]:
(1-(0.95)**6) * 20

5.2981621875000044

In [1]:
from sympy import symbols, integrate, exp, oo

# 定義する変数
x = symbols('x')

# 関数の定義
f1 = (3/32) * x**2  # 0 <= x < 2 の範囲
f2 = (3/8) * exp(1 - x/2)  # x >= 2 の範囲

# 期待値の積分
expectation_1 = integrate(x * f1, (x, 0, 2))  # 0から2までの積分
expectation_2 = integrate(x * f2, (x, 2, oo))  # 2から無限大までの積分

# 期待値の合計
expected_value = expectation_1 + expectation_2
expected_value.evalf()  # 数値を計算


3.37500000000000

In [2]:
import sympy as sp

# 定義
x = sp.Symbol('x')
f1 = 3/32 * x**2  # 0 ≤ x < 2 の部分
f2 = 3/8 * sp.exp(1 - x/2)  # x ≥ 2 の部分

# 確率密度関数を積分して累積分布関数 F(T) を求める
F1 = sp.integrate(f1, (x, 0, 2))  # 0 から 2 までの積分
F2 = sp.integrate(f2, (x, 2, x))  # 2 から x までの積分 (一般形)

# 総累積分布関数
F = F1 + F2

# 保証期間を求めるための方程式
T = sp.Symbol('T', positive=True)
p = sp.solve(F.subs(x, T) - 1/6, T)  # 故障率 1/6 に基づいた保証期間の解を求める
p


[1.78927896868435]